In [1]:
from scipy.stats import rankdata
import scipy as sp
import numpy as np
import pandas as pd

### 读取数据

In [2]:
df = pd.read_csv('./alldata.csv')

D:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### 辅助函数

In [3]:
def get_data(column):
    df_res = df.pivot(index=['Trddt'], columns='Stkcd', values=[column])
    df_res = df_res[column]
    df_res = df_res.reset_index()
    df_res.fillna(df_res.mean(), inplace=True)
    df_res = df_res.set_index('Trddt')
    def change_extreme(row):
        m=row.mean()
        sd=row.std()
        row=row.apply(lambda x:x if m-3*sd<=x<=m+3*sd else (m+3*sd if x>m+3*sd else m-3*sd))
        return row
    df_res = df_res.apply(change_extreme, axis=1)
    df_res = df_res.apply(lambda row: (row-row.mean())/row.std(), axis=1)
    return df_res
                    

In [4]:
import statsmodels.api as sm 
def compute_IC_t(df,return_nextday):#df为因子值矩阵
    
    #行列对齐
    return_nextday=return_nextday[df.columns.tolist()]
    return_nextday=return_nextday.loc[df.index.tolist()]
    
    #建立新的dataframe，用于存储结果
    result=pd.DataFrame(np.nan,index=df.index,columns=["factor_return","tvalue","IC"])

    #T+1期收益率对T期因子值逐行做回归
    for date in df.index.tolist():
        x=df.loc[date]
        y=return_nextday.loc[date]
        model=sm.OLS(y,x).fit()
        result.loc[date,"factor_return"]=model.params[date]
        result.loc[date,"tvalue"]=model.tvalues[date]
        result.loc[date,"IC"]=return_nextday.loc[date].corr(df.loc[date])
    
    return result

In [5]:
df_high = get_data('Hiprc')
df_low = get_data('Loprc')
df_open = get_data('Opnprc')
df_close = get_data('Clsprc')
df_volume = get_data('Dnshrtrd')
df_value = get_data('Dnvaltrd')
df_marketcap = get_data('Dsmvosd')
df_low

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,-0.073474,-0.604459,-0.270474,-0.794851,-0.583756,-0.754662,-0.687357,-0.596340,-0.556151,-0.603242,...,-1.000831,-0.683214,-1.006596,-1.002090,-1.013010,-1.015405,-1.005621,-1.006149,-1.002536,-1.012604
2010-01-05,-0.110619,-0.619510,-0.269685,-0.796710,-0.600858,-0.756161,-0.686094,-0.608968,-0.565985,-0.619105,...,-0.999496,-0.677982,-1.004930,-1.001362,-1.011377,-1.013729,-1.004119,-1.005011,-1.000875,-1.011175
2010-01-06,-0.114515,-0.622210,-0.272371,-0.790090,-0.601123,-0.747106,-0.688796,-0.615316,-0.567466,-0.625859,...,-1.001845,-0.674723,-1.007523,-1.002900,-1.014011,-1.016322,-1.006022,-1.007806,-1.003346,-1.013848
2010-01-07,-0.123432,-0.616368,-0.268264,-0.795544,-0.601775,-0.750953,-0.684554,-0.614747,-0.580290,-0.625692,...,-0.998191,-0.675715,-1.003542,-0.998434,-1.010352,-1.012257,-1.001434,-1.003258,-0.999529,-1.010190
2010-01-08,-0.122881,-0.616447,-0.265869,-0.797070,-0.597776,-0.752827,-0.682691,-0.601835,-0.582352,-0.623753,...,-0.996972,-0.677331,-1.002249,-0.996972,-1.009027,-1.010894,-1.000016,-1.001640,-0.998393,-1.009108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.196563,-0.058285,-0.068324,-0.678432,-0.603626,-0.526877,-0.662240,-0.296629,-0.630180,-0.380179,...,-0.737500,-0.717390,-0.727444,-0.722956,-0.727116,-0.738536,-0.735654,-0.742649,-0.708161,-0.726522
2022-01-25,-0.190111,-0.063019,-0.083444,-0.668653,-0.596029,-0.517893,-0.653415,-0.294833,-0.625208,-0.381722,...,-0.726363,-0.706521,-0.716101,-0.711608,-0.715773,-0.727271,-0.724580,-0.731453,-0.696796,-0.715437
2022-01-26,-0.193306,-0.063293,-0.005510,-0.669693,-0.596151,-0.523593,-0.653278,-0.305591,-0.627012,-0.377820,...,-0.727444,-0.707384,-0.717085,-0.712536,-0.716753,-0.728331,-0.725606,-0.732533,-0.697536,-0.716314


In [6]:
df_return = df_close / df_close.shift(1) - 1
df_return

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.197883,0.018556,0.007975,-0.008275,0.024576,-0.011253,0.002644,0.021485,0.018096,0.024434,...,0.001490,-0.014547,0.001202,0.001013,0.001268,0.001341,0.000770,0.001951,0.001091,0.001699
2010-01-06,0.137473,-0.000617,-0.001536,0.005551,-0.002595,-0.000499,-0.000552,0.000016,-0.008936,-0.002511,...,-0.000312,0.002063,-0.000389,-0.000865,-0.000151,-0.000229,-0.000941,-0.000861,-0.000312,-0.000503
2010-01-07,0.045094,-0.001980,-0.015590,0.006812,-0.001371,0.005129,-0.006445,-0.019972,0.024258,0.008157,...,-0.003919,0.002387,-0.004098,-0.004196,-0.003919,-0.004301,-0.004223,-0.004452,-0.003955,-0.003841
2010-01-08,0.023834,-0.001222,0.005318,-0.000088,-0.013768,0.000332,0.003095,-0.030837,-0.010239,-0.003078,...,0.002410,-0.003818,0.002564,0.002410,0.002401,0.002439,0.002447,0.002288,0.002687,0.002637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,0.025036,-0.013422,-0.120129,-0.004414,-0.001084,-0.008103,-0.005321,-0.030517,-0.000293,0.005083,...,-0.003861,-0.004107,-0.004091,-0.004078,-0.004089,-0.003949,-0.003941,-0.004045,-0.004037,-0.003351
2022-01-25,-0.046020,0.072399,0.220690,-0.010743,-0.009565,-0.018687,-0.007733,0.019286,-0.005615,0.000166,...,-0.010816,-0.010994,-0.011593,-0.011816,-0.011617,-0.011028,-0.010985,-0.010801,-0.012539,-0.011191
2022-01-26,0.045081,0.084356,-0.738770,0.001022,-0.000139,0.023897,-0.001834,0.033689,-0.000258,-0.017911,...,0.001169,0.000960,0.001226,0.001252,0.001228,0.000988,0.001179,0.001099,0.001336,0.001094


### 因子

In [7]:
simple_factors = [df_high, df_low, df_open, df_close, df_volume, df_value, df_marketcap]

### 因子回归结果

In [8]:
res = []
for i in simple_factors:
    res.append(compute_IC_t(i.shift(2)[2:], df_return))
res[0]

,factor_return,tvalue,IC
Trddt,,,
2010-01-06,-0.000283,-0.054351,-0.000783
2010-01-07,0.003851,0.174717,0.002516
2010-01-08,0.000359,0.026856,0.000387
2010-01-11,0.005526,0.123830,0.001783
2010-01-12,0.008972,0.293352,0.004224
...,...,...,...
2022-01-24,0.002208,0.028345,0.000408
2022-01-25,-0.002376,-0.006312,-0.000091
2022-01-26,-0.000621,-0.034176,-0.000492


In [9]:
res[1]

,factor_return,tvalue,IC
Trddt,,,
2010-01-06,-0.000400,-0.076956,-0.001108
2010-01-07,0.003736,0.169527,0.002441
2010-01-08,0.000415,0.030998,0.000446
2010-01-11,0.005434,0.121757,0.001753
2010-01-12,0.008606,0.281380,0.004052
...,...,...,...
2022-01-24,0.002996,0.038458,0.000554
2022-01-25,-0.000634,-0.001683,-0.000024
2022-01-26,-0.000920,-0.050659,-0.000729


# 其他因子

<ol>
    <li>
        收益率比五天前收益率 <b>(不显著)</b>
        <code>((df_return / df_return.shift(5)) - 1).shift(2)[8:]</code>
    </li>
    <li>
        五日反转因子
        <code>((df_close / df_close.shift(5)) - 1)[5:]</code>
    </li>
    <li>
        开/收 开/高 等等（6个）
        <pre>
            <code>
            price = [df_high, df_low, df_open, df_close]
            res = []
            for i in range(4):
                for j in range(i+1, 4):
                    res.append(get_t_value(price[i]/price[j]))
            </code>
        </pre>
    </li>
    <li>
        单个数据因子（7个）
    </li>
    <li>
        成交额比市值 <b>(不显著)</b> <code>df_value/df_marketcap</code>
    </li>
</ol>

In [10]:
((df_close / df_close.shift(5)) - 1)[5:]

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-11,0.492245,0.027981,0.006166,0.008419,0.028398,-0.007156,0.000481,-0.012758,0.020140,0.011913,...,-0.000744,-0.017873,-0.000875,-0.001141,-0.000499,-0.000817,-0.001227,-0.000796,-0.000550,0.000595
2010-01-12,0.345805,0.011685,0.018278,0.008995,-0.017262,0.007946,0.006466,-0.042031,-0.011272,-0.018027,...,0.003240,0.001680,0.003884,0.003908,0.004008,0.003685,0.003773,0.003293,0.004300,0.004554
2010-01-13,0.631221,0.024296,0.012855,0.004596,0.004906,0.015034,0.003419,-0.007401,-0.000165,0.016046,...,0.000875,0.006214,0.001562,0.002610,0.000989,0.001059,0.002554,0.001445,0.002013,0.002199
2010-01-14,0.603511,0.036525,0.051651,0.003715,0.012233,0.008805,0.018618,0.013550,-0.022400,0.009135,...,0.010601,0.009809,0.011126,0.012782,0.010527,0.011088,0.012180,0.011447,0.010389,0.011273
2010-01-15,0.445139,0.033353,0.052193,0.004763,0.015120,0.000068,0.017768,0.010082,-0.016362,-0.004088,...,0.009619,0.015414,0.009625,0.011636,0.008871,0.010053,0.010875,0.010539,0.008219,0.009690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.227344,-0.498251,-0.046149,-0.007557,-0.020470,-0.042302,-0.014147,-0.067152,-0.008663,-0.044689,...,-0.011755,-0.013886,-0.012647,-0.012973,-0.012682,-0.011642,-0.011595,-0.011572,-0.014033,-0.011732
2022-01-25,-0.215068,-0.293336,0.651310,-0.015118,-0.020365,-0.038321,-0.012743,-0.005591,-0.010222,-0.033214,...,-0.014964,-0.017088,-0.016537,-0.017022,-0.016588,-0.015197,-0.014987,-0.014914,-0.018595,-0.015163
2022-01-26,-0.176841,-0.102510,-0.698848,-0.025458,-0.025874,-0.040631,-0.018109,-0.011813,-0.016777,-0.031364,...,-0.022763,-0.024766,-0.024135,-0.024546,-0.024179,-0.023142,-0.022880,-0.022791,-0.025878,-0.022843


In [11]:
df_res = compute_IC_t((df_high / df_low).shift(2)[2:], df_return)

In [12]:
df_res.mean()

factor_return   -0.021492
tvalue           0.043314
IC               0.000558
dtype: float64

In [13]:
df_res[df_res['factor_return'] < 0].mean()

factor_return   -0.088108
tvalue          -2.131125
IC              -0.037603
dtype: float64

In [14]:
df_res = compute_IC_t((df_return / df_return.shift(5)).shift(2)[8:], df_return)

In [15]:
df_res

,factor_return,tvalue,IC
Trddt,,,
2010-01-14,0.000004,0.011700,-0.000127
2010-01-15,-0.000005,-0.047733,-0.000597
2010-01-18,-0.000254,-0.410217,-0.005873
2010-01-19,-0.002833,-3.224405,-0.046534
2010-01-20,-0.000105,-0.072993,-0.001069
...,...,...,...
2022-01-24,-0.000058,-0.094463,-0.001853
2022-01-25,0.000023,0.017117,-0.000013
2022-01-26,0.000047,0.105373,0.001518


In [16]:
def get_t_value(df):
    df_res = compute_IC_t(df, df_return)
    one = df_res['factor_return'].mean()
    two = abs(df_res['tvalue']).mean()
    three = len(df_res[df_res['tvalue'] > 2]) / len(df_res)
    four = df_res[df_res['tvalue']>0].mean()['tvalue']
    five = df_res[df_res['tvalue']<0].mean()['tvalue']
    six = df_res[df_res['IC']>0].mean()['IC']
    seven = df_res[df_res['IC']<0].mean()['IC']
    df_res = pd.DataFrame(data={'平均收益率':[one],'t值绝对值均值':[two],'t值大于2的占比':[three],'正t值均值':[four]
                                ,'负t值均值':[five],'正ic值均值':[six],'负ic值均值':[seven]})
    return df_res

In [17]:
get_t_value(((df_close / df_close.shift(5)) - 1)[5:])

,平均收益率,t值绝对值均值,t值大于2的占比,正t值均值,负t值均值,正ic值均值,负ic值均值
0,0.000071,4.252526,0.238144,3.830394,-4.929211,0.051108,-0.049351


In [18]:
price = [df_high, df_low, df_open, df_close]
res = []
for i in range(4):
    for j in range(i+1, 4):
        res.append(get_t_value(price[i]/price[j]))

In [19]:
res[1]

,平均收益率,t值绝对值均值,t值大于2的占比,正t值均值,负t值均值,正ic值均值,负ic值均值
0,-0.019763,7.845864,0.427793,7.790122,-7.933759,0.151326,-0.129989


In [20]:
get_t_value(df_value/df_marketcap)

,平均收益率,t值绝对值均值,t值大于2的占比,正t值均值,负t值均值,正ic值均值,负ic值均值
0,-0.000046,0.316539,0.007153,0.273983,-0.359884,0.003862,-0.004591


In [21]:
df_reverse = ((df_close / df_close.shift(5)) - 1)[5:]
df_reverse

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-11,0.492245,0.027981,0.006166,0.008419,0.028398,-0.007156,0.000481,-0.012758,0.020140,0.011913,...,-0.000744,-0.017873,-0.000875,-0.001141,-0.000499,-0.000817,-0.001227,-0.000796,-0.000550,0.000595
2010-01-12,0.345805,0.011685,0.018278,0.008995,-0.017262,0.007946,0.006466,-0.042031,-0.011272,-0.018027,...,0.003240,0.001680,0.003884,0.003908,0.004008,0.003685,0.003773,0.003293,0.004300,0.004554
2010-01-13,0.631221,0.024296,0.012855,0.004596,0.004906,0.015034,0.003419,-0.007401,-0.000165,0.016046,...,0.000875,0.006214,0.001562,0.002610,0.000989,0.001059,0.002554,0.001445,0.002013,0.002199
2010-01-14,0.603511,0.036525,0.051651,0.003715,0.012233,0.008805,0.018618,0.013550,-0.022400,0.009135,...,0.010601,0.009809,0.011126,0.012782,0.010527,0.011088,0.012180,0.011447,0.010389,0.011273
2010-01-15,0.445139,0.033353,0.052193,0.004763,0.015120,0.000068,0.017768,0.010082,-0.016362,-0.004088,...,0.009619,0.015414,0.009625,0.011636,0.008871,0.010053,0.010875,0.010539,0.008219,0.009690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.227344,-0.498251,-0.046149,-0.007557,-0.020470,-0.042302,-0.014147,-0.067152,-0.008663,-0.044689,...,-0.011755,-0.013886,-0.012647,-0.012973,-0.012682,-0.011642,-0.011595,-0.011572,-0.014033,-0.011732
2022-01-25,-0.215068,-0.293336,0.651310,-0.015118,-0.020365,-0.038321,-0.012743,-0.005591,-0.010222,-0.033214,...,-0.014964,-0.017088,-0.016537,-0.017022,-0.016588,-0.015197,-0.014987,-0.014914,-0.018595,-0.015163
2022-01-26,-0.176841,-0.102510,-0.698848,-0.025458,-0.025874,-0.040631,-0.018109,-0.011813,-0.016777,-0.031364,...,-0.022763,-0.024766,-0.024135,-0.024546,-0.024179,-0.023142,-0.022880,-0.022791,-0.025878,-0.022843


In [22]:
df_reverse.shift(1)

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,0.492245,0.027981,0.006166,0.008419,0.028398,-0.007156,0.000481,-0.012758,0.020140,0.011913,...,-0.000744,-0.017873,-0.000875,-0.001141,-0.000499,-0.000817,-0.001227,-0.000796,-0.000550,0.000595
2010-01-13,0.345805,0.011685,0.018278,0.008995,-0.017262,0.007946,0.006466,-0.042031,-0.011272,-0.018027,...,0.003240,0.001680,0.003884,0.003908,0.004008,0.003685,0.003773,0.003293,0.004300,0.004554
2010-01-14,0.631221,0.024296,0.012855,0.004596,0.004906,0.015034,0.003419,-0.007401,-0.000165,0.016046,...,0.000875,0.006214,0.001562,0.002610,0.000989,0.001059,0.002554,0.001445,0.002013,0.002199
2010-01-15,0.603511,0.036525,0.051651,0.003715,0.012233,0.008805,0.018618,0.013550,-0.022400,0.009135,...,0.010601,0.009809,0.011126,0.012782,0.010527,0.011088,0.012180,0.011447,0.010389,0.011273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,-0.211163,-0.467659,-0.081743,-0.002372,-0.019211,-0.038194,-0.006381,-0.024273,-0.006300,-0.055857,...,-0.007784,-0.009289,-0.008234,-0.008471,-0.008259,-0.007646,-0.007499,-0.007528,-0.009238,-0.008016
2022-01-25,-0.227344,-0.498251,-0.046149,-0.007557,-0.020470,-0.042302,-0.014147,-0.067152,-0.008663,-0.044689,...,-0.011755,-0.013886,-0.012647,-0.012973,-0.012682,-0.011642,-0.011595,-0.011572,-0.014033,-0.011732
2022-01-26,-0.215068,-0.293336,0.651310,-0.015118,-0.020365,-0.038321,-0.012743,-0.005591,-0.010222,-0.033214,...,-0.014964,-0.017088,-0.016537,-0.017022,-0.016588,-0.015197,-0.014987,-0.014914,-0.018595,-0.015163


In [26]:
df_reverse_rank = df_reverse.rank(ascending=False)
df_reverse_rank[df_reverse_rank<=100] 

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-11,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-13,51.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-14,57.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-15,81.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-01-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
(df_return.shift(-1) * df_reverse_rank[df_reverse_rank<=100]).sum(axis=1)

Trddt
2010-01-04      0.000000
2010-01-05      0.000000
2010-01-06      0.000000
2010-01-07      0.000000
2010-01-08      0.000000
                 ...    
2022-01-24     76.093767
2022-01-25   -105.128659
2022-01-26    336.610682
2022-01-27   -205.983521
2022-01-28      0.000000
Length: 2936, dtype: float64

In [36]:
df_return

Stkcd,1,2,4,5,6,7,8,9,10,11,...,900947,900948,900949,900950,900951,900952,900953,900955,900956,900957
Trddt,,,,,,,,,,,,,,,,,,,,,
2010-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-01-05,0.197883,0.018556,0.007975,-0.008275,0.024576,-0.011253,0.002644,0.021485,0.018096,0.024434,...,0.001490,-0.014547,0.001202,0.001013,0.001268,0.001341,0.000770,0.001951,0.001091,0.001699
2010-01-06,0.137473,-0.000617,-0.001536,0.005551,-0.002595,-0.000499,-0.000552,0.000016,-0.008936,-0.002511,...,-0.000312,0.002063,-0.000389,-0.000865,-0.000151,-0.000229,-0.000941,-0.000861,-0.000312,-0.000503
2010-01-07,0.045094,-0.001980,-0.015590,0.006812,-0.001371,0.005129,-0.006445,-0.019972,0.024258,0.008157,...,-0.003919,0.002387,-0.004098,-0.004196,-0.003919,-0.004301,-0.004223,-0.004452,-0.003955,-0.003841
2010-01-08,0.023834,-0.001222,0.005318,-0.000088,-0.013768,0.000332,0.003095,-0.030837,-0.010239,-0.003078,...,0.002410,-0.003818,0.002564,0.002410,0.002401,0.002439,0.002447,0.002288,0.002687,0.002637
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-24,0.025036,-0.013422,-0.120129,-0.004414,-0.001084,-0.008103,-0.005321,-0.030517,-0.000293,0.005083,...,-0.003861,-0.004107,-0.004091,-0.004078,-0.004089,-0.003949,-0.003941,-0.004045,-0.004037,-0.003351
2022-01-25,-0.046020,0.072399,0.220690,-0.010743,-0.009565,-0.018687,-0.007733,0.019286,-0.005615,0.000166,...,-0.010816,-0.010994,-0.011593,-0.011816,-0.011617,-0.011028,-0.010985,-0.010801,-0.012539,-0.011191
2022-01-26,0.045081,0.084356,-0.738770,0.001022,-0.000139,0.023897,-0.001834,0.033689,-0.000258,-0.017911,...,0.001169,0.000960,0.001226,0.001252,0.001228,0.000988,0.001179,0.001099,0.001336,0.001094


In [41]:
df_return.shift(-1)[df_reverse_rank<=100].mean(axis=1).sum()

-36.3312847606322

In [46]:
def choose(df, rev: bool = True, top: int = 100):
    """
    df: 因子值矩阵
    返回每天收益率序列
    """
    df = df.shift(1)
    df_rank = df.rank(ascending=rev)
    return df_return.shift(-1)[df_rank<=top].mean(axis=1)

In [48]:
df_open_r = choose(df_open)

In [54]:
df_open_r

Trddt
2010-01-04         NaN
2010-01-05    0.006507
2010-01-06    0.032941
2010-01-07   -0.010789
2010-01-08   -0.011470
                ...   
2022-01-24   -0.001779
2022-01-25   -0.008710
2022-01-26   -0.103901
2022-01-27    0.012170
2022-01-28         NaN
Length: 2936, dtype: float64

In [53]:
df_open_r.mean()

-0.0031748991748651936

In [77]:
def MaxDrawdown(return_list):
    '''最大回撤率'''
    i = np.argmax((np.maximum.accumulate(return_list) - return_list) / np.maximum.accumulate(return_list))  # 结束位置
    if i == 0:
        return 0
    j = np.argmax(return_list[:i])  # 开始位置
    return (return_list[j] - return_list[i]) / (return_list[j]) 
MaxDrawdown(ret)

27.81805583658606

In [78]:
from math import sqrt
def evaluate(df):
    sharpe = df.mean() / df.std() * sqrt(250)
    total = df.sum()
    max_drawdown = MaxDrawdown(list(df.dropna()))
    return sharpe, total, max_drawdown

In [79]:
def all_choose(df, rev: bool = True, top: int = 100):
    df_c = choose(df, rev, top)
    s, t, m = evaluate(df_c)
    df_res = pd.DataFrame(data={'夏普':[s],'累计收益率':[t],'最大回撤':[m]})
    return df_res

In [83]:
all_choose(df_close, True, 500)

,夏普,累计收益率,最大回撤
0,-0.128665,-8.120496,22.437052


In [85]:
np.maximum.accumulate([1,2,3,0,-2,7,-2]) - [1,2,3,0,-2,7,-2]

array([0, 0, 0, 3, 5, 0, 9])